# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [89]:
import pandas as pd
import numpy as np


class Neural_Network(object):
    def __init__(self, inputs=3, hiddenNodes=3, outputNodes=1):
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes

        # Initialize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

    def sigmoid(self, s):
        return 1/(1+np.exp(-s))

    def sigmoidPrime(self, s):
        """derivative of sigmoid"""
        return s * (1 - s)

    def backward(self, X, y, o):
        """backward propgate through the network"""
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
  
        self.z2_error = self.o_delta.dot(self.L2_weights.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
  
        self.L1_weights += X.T.dot(self.z2_delta)
        self.L2_weights += self.activated_hidden.T.dot(self.o_delta)

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [90]:
df = pd.DataFrame({
    'x1':[0, 0, 1, 0, 1, 1, 0],
    'x2':[0, 1, 0, 1, 0, 1, 0],
    'x3':[1, 1, 1, 0, 0, 1, 0],
    'y': [0, 1, 1, 1, 1, 0, 0]
})
X = df.drop(columns='y').values
y = df[['y']].values

nn = Neural_Network(inputs=3, hiddenNodes=4, outputNodes=1)

In [91]:
nn.feed_forward(X[0])

array([0.34908627])

In [92]:
for i in range(1000):
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('EPOCH', i+1, "Loss: \n", 
              str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

EPOCH 1 Loss: 
 0.33210718765303043
EPOCH 2 Loss: 
 0.2972578227046813
EPOCH 3 Loss: 
 0.2743965742500592
EPOCH 4 Loss: 
 0.2613579279961759
EPOCH 5 Loss: 
 0.25415929213607635
EPOCH 50 Loss: 
 0.17206640127689538
EPOCH 100 Loss: 
 0.1285208959123444
EPOCH 150 Loss: 
 0.11415431657670912
EPOCH 200 Loss: 
 0.103597071428868
EPOCH 250 Loss: 
 0.08778125824048862
EPOCH 300 Loss: 
 0.05548944803905013
EPOCH 350 Loss: 
 0.031165162786199876
EPOCH 400 Loss: 
 0.020140001247423482
EPOCH 450 Loss: 
 0.014412658523868012
EPOCH 500 Loss: 
 0.01100179121029593
EPOCH 550 Loss: 
 0.00880999599284309
EPOCH 600 Loss: 
 0.007321449683109214
EPOCH 650 Loss: 
 0.006257303259174124
EPOCH 700 Loss: 
 0.005461525164289533
EPOCH 750 Loss: 
 0.004844371244385743
EPOCH 800 Loss: 
 0.0043517653165920265
EPOCH 850 Loss: 
 0.003949412101563925
EPOCH 900 Loss: 
 0.003614544715006361
EPOCH 950 Loss: 
 0.003331460922222088
EPOCH 1000 Loss: 
 0.003088981723433024


## Try building/training a more complex Multilayer Perceptron on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [93]:
import gzip
import numpy as np
import pandas as pd


def load_images(num_images):
    # training images
    f = gzip.open('datasets/train-images-idx3-ubyte.gz','r')

    image_size = 28

    f.read(16)
    buf = f.read(image_size * image_size * num_images)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = data.reshape(num_images, image_size, image_size, 1)
    
    return data


def load_labels(n_labels):
    f = gzip.open('datasets/train-labels-idx1-ubyte.gz','r')
    f.read(8)
    l = []
    for i in range(0,n_labels):   
        buf = f.read(1)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        l.append(labels[0])
        
    return l

In [108]:
X = load_images(num_images=1000)
X = X.reshape(X.shape[0], 28*28)
y = np.array(load_labels(n_labels=1000)).reshape(1000, 1)
# TODO target encoding

In [ ]:
nn = Neural_Network(inputs=X.shape[1], hiddenNodes=4, outputNodes=1)

for i in range(1000):
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('EPOCH', i+1, "Loss: \n", 
              str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?